In [2]:
%run "../layeringOfServices.ipynb"

In [7]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Old hts communes 

In [4]:
hts_5Mai2021 = pd.read_excel("hts_5Mai2021.xlsx")

In [8]:
hts_5Mai2021.date_entevyou.min().strftime("%Y-%m-%d")
hts_5Mai2021.date_entevyou.max().strftime("%Y-%m-%d")

'2020-04-07'

'2021-04-10'

In [9]:
hts_5Mai2021.actual_commune.unique()

array(['Dessalines', 'Kenscoff', 'Gressier', 'Cap-Haïtien', 'Milot',
       'Plaine-du-Nord', 'Quartier-Morin', 'Limonade', 'Verrettes',
       'Liancourt', 'Carrefour', 'Delmas', 'Pétionville', 'Saint-Marc',
       'La Chapelle', 'Port-au-Prince', "Petite Rivière de l'Artibonite",
       'Grande Saline', 'Tabarre', 'Montrouis', 'Desdunes'], dtype=object)

In [10]:
from re import sub
def name_handler(s):
    s = sub(r"[^\w\s]", '', s)
    # Replace all runs of whitespace with a single dash
    s = sub(r"\s+", '_', s)
    return s



In [12]:
hts_5Mai2021_communes = []
for columns in hts_5Mai2021_communes.unique():
    globals()[f"hts5Juin_{name_handler(columns)}"] = hts_5Mai2021[hts_5Mai2021.actual_commune == columns]
    hts_5Mai2021_communes.append(globals().get(f"hts5Juin_{name_handler(columns)}"))